# Infer-2-Gaussian-Mixtures : Distributions Gaussiennes et Melanges

**Serie** : Programmation Probabiliste avec Infer.NET (2/12)  
**Duree estimee** : 50 minutes  
**Prerequis** : Infer-1-Setup

---

## Objectifs

- Maitriser les distributions Gaussienne et Gamma
- Comprendre la notion de priors conjugues
- Implementer l'apprentissage de parametres
- Construire des modeles de melange de Gaussiennes
- Utiliser `Variable.Switch` pour les modeles a composantes

---

## Navigation

| Precedent | Suivant |
|-----------|--------|
| [Infer-1-Setup](Infer-1-Setup.ipynb) | [Infer-3-Factor-Graphs](Infer-3-Factor-Graphs.ipynb) |

---

## 1. Configuration

In [ ]:
#r "nuget: Microsoft.ML.Probabilistic"
#r "nuget: Microsoft.ML.Probabilistic.Compiler"

using Microsoft.ML.Probabilistic;
using Microsoft.ML.Probabilistic.Distributions;
using Microsoft.ML.Probabilistic.Utilities;
using Microsoft.ML.Probabilistic.Math;
using Microsoft.ML.Probabilistic.Models;
using Microsoft.ML.Probabilistic.Algorithms;
using Microsoft.ML.Probabilistic.Compiler;

Console.WriteLine("Infer.NET pret !");

## 2. Scenario : Le Cycliste

### Contexte

Vous vous rendez au travail a velo chaque jour. Votre temps de trajet varie :
- Certains jours, le trajet est rapide (peu de trafic)
- D'autres jours, il est plus lent (embouteillages, meteo)

### Objectifs

1. **Apprendre** la distribution du temps de trajet a partir d'observations
2. **Predire** le temps de trajet de demain
3. **Calculer** des probabilites (ex: P(trajet < 18 min))

### Modelisation

Nous modelerons le temps de trajet avec une distribution **Gaussienne** :

$$\text{temps} \sim \mathcal{N}(\mu, \tau^{-1})$$

- $\mu$ : moyenne (temps moyen de trajet)
- $\tau$ : precision (inverse de la variance)

## 3. Distributions Conjuguees

### Theorie

En inference bayesienne, une distribution **a priori conjuguee** permet une mise a jour analytique simple :

| Vraisemblance | Prior Conjugue | Posterieur |
|---------------|----------------|------------|
| Gaussian (moyenne) | Gaussian | Gaussian |
| Gaussian (precision) | Gamma | Gamma |
| Bernoulli | Beta | Beta |
| Discrete | Dirichlet | Dirichlet |

### Pour notre modele cycliste

- **Moyenne** : prior Gaussian vague $\mu \sim \mathcal{N}(15, 100)$
- **Precision** : prior Gamma $\tau \sim \text{Gamma}(2, 0.5)$

## 4. Modele Simple : Une Gaussienne

In [ ]:
// Definition du modele
// Prior sur la moyenne : Gaussian vague centree sur 15 min
Variable<double> dureeMoyenne = Variable.GaussianFromMeanAndPrecision(15, 0.01);  // precision = 0.01 -> variance = 100

// Prior sur la precision (inverse de la variance)
Variable<double> bruitTrafic = Variable.GammaFromShapeAndScale(2, 0.5);

In [ ]:
// Definition des temps de trajet observes
Variable<double> dureeLundi = Variable.GaussianFromMeanAndPrecision(dureeMoyenne, bruitTrafic);
Variable<double> dureeMardi = Variable.GaussianFromMeanAndPrecision(dureeMoyenne, bruitTrafic);
Variable<double> dureeMercredi = Variable.GaussianFromMeanAndPrecision(dureeMoyenne, bruitTrafic);

// Observations
dureeLundi.ObservedValue = 13;
dureeMardi.ObservedValue = 17;
dureeMercredi.ObservedValue = 16;

In [ ]:
// Inference
InferenceEngine moteur = new InferenceEngine();
moteur.Compiler.CompilerChoice = CompilerChoice.Roslyn;

Gaussian moyennePosterieure = moteur.Infer<Gaussian>(dureeMoyenne);
Gamma bruitPosterieur = moteur.Infer<Gamma>(bruitTrafic);

Console.WriteLine($"Moyenne a posteriori : {moyennePosterieure}");
Console.WriteLine($"Precision a posteriori : {bruitPosterieur}");
Console.WriteLine($"\nMoyenne estimee : {moyennePosterieure.GetMean():F2} min");
Console.WriteLine($"Ecart-type du bruit : {Math.Sqrt(1/bruitPosterieur.GetMean()):F2} min");

## 5. Prediction du Temps de Demain

In [ ]:
// Prediction pour demain
Variable<double> dureeDemain = Variable.GaussianFromMeanAndPrecision(dureeMoyenne, bruitTrafic);
Gaussian distribDemain = moteur.Infer<Gaussian>(dureeDemain);

Console.WriteLine($"Prediction demain : {distribDemain}");
Console.WriteLine($"Temps moyen estime : {distribDemain.GetMean():F2} min");
Console.WriteLine($"Ecart-type de la prediction : {Math.Sqrt(distribDemain.GetVariance()):F2} min");

// Intervalle de confiance a 95%
double mean = distribDemain.GetMean();
double std = Math.Sqrt(distribDemain.GetVariance());
Console.WriteLine($"\nIntervalle de confiance 95% : [{mean - 1.96*std:F1}, {mean + 1.96*std:F1}] min");

## 6. Calcul de Probabilites

In [ ]:
// Quelle est la probabilite que le trajet dure moins de 18 minutes ?
Bernoulli probMoinsDe18 = moteur.Infer<Bernoulli>(dureeDemain < 18.0);
Console.WriteLine($"P(trajet < 18 min) = {probMoinsDe18.GetProbTrue():F2}");

// Et moins de 15 minutes ?
Bernoulli probMoinsDe15 = moteur.Infer<Bernoulli>(dureeDemain < 15.0);
Console.WriteLine($"P(trajet < 15 min) = {probMoinsDe15.GetProbTrue():F2}");

// Entre 14 et 18 minutes ?
Variable<bool> entre14et18 = (dureeDemain > 14.0) & (dureeDemain < 18.0);
Bernoulli probEntre = moteur.Infer<Bernoulli>(entre14et18);
Console.WriteLine($"P(14 < trajet < 18 min) = {probEntre.GetProbTrue():F2}");

## 7. Restructuration avec Classes

Pour des modeles plus complexes, il est preferable d'encapsuler le code dans des classes reutilisables.

In [ ]:
// Structure pour stocker les posterieurs
public struct DonneesCycliste
{
    public Gaussian DistribMoyenne;
    public Gamma DistribBruitTraffic;
    
    public DonneesCycliste(Gaussian moyenne, Gamma precision)
    {
        DistribMoyenne = moyenne;
        DistribBruitTraffic = precision;
    }
}

// Classe de base avec les elements communs
public class CyclisteBase
{
    public InferenceEngine MoteurInference;
    protected Variable<double> Moyenne;
    protected Variable<double> Bruit;
    protected Variable<Gaussian> MoyenneAPriori;
    protected Variable<Gamma> BruitAPriori;

    public virtual void CreationModeleBayesien()
    {
        MoyenneAPriori = Variable.New<Gaussian>();
        BruitAPriori = Variable.New<Gamma>();
        Moyenne = Variable.Random<double, Gaussian>(MoyenneAPriori);
        Bruit = Variable.Random<double, Gamma>(BruitAPriori);

        if (MoteurInference == null)
        {
            MoteurInference = new InferenceEngine(new ExpectationPropagation());
            MoteurInference.Compiler.CompilerChoice = CompilerChoice.Roslyn;
        }
    }

    public virtual void DefinirDistributions(DonneesCycliste distribsApriori)
    {
        MoyenneAPriori.ObservedValue = distribsApriori.DistribMoyenne;
        BruitAPriori.ObservedValue = distribsApriori.DistribBruitTraffic;
    }
}

Console.WriteLine("Classe CyclisteBase definie.");

In [ ]:
// Classe d'entrainement
public class EntrainementCycliste : CyclisteBase
{
    protected VariableArray<double> TempsDeTrajet;
    protected Variable<int> NombreDeTrajets;

    public override void CreationModeleBayesien()
    {
        base.CreationModeleBayesien();
        NombreDeTrajets = Variable.New<int>();
        Range indiceTrajet = new Range(NombreDeTrajets);
        TempsDeTrajet = Variable.Array<double>(indiceTrajet);
        using (Variable.ForEach(indiceTrajet))
        {
            TempsDeTrajet[indiceTrajet] = Variable.GaussianFromMeanAndPrecision(Moyenne, Bruit);
        }
    }

    public DonneesCycliste CalculePosterieurs(double[] donneesObservees)
    {
        DonneesCycliste posterieurs;
        NombreDeTrajets.ObservedValue = donneesObservees.Length;
        TempsDeTrajet.ObservedValue = donneesObservees;
        posterieurs.DistribMoyenne = MoteurInference.Infer<Gaussian>(Moyenne);
        posterieurs.DistribBruitTraffic = MoteurInference.Infer<Gamma>(Bruit);
        return posterieurs;
    }
}

// Classe de prediction
public class PredictionCycliste : CyclisteBase
{
    public Variable<double> demainTemps;

    public override void CreationModeleBayesien()
    {
        base.CreationModeleBayesien();
        demainTemps = Variable.GaussianFromMeanAndPrecision(Moyenne, Bruit);
    }

    public Gaussian EstimerTempsDemain()
    {
        return MoteurInference.Infer<Gaussian>(demainTemps);
    }

    public Bernoulli EstimerTempsDemainInferieurA(double duree)
    {
        return MoteurInference.Infer<Bernoulli>(demainTemps < duree);
    }
}

Console.WriteLine("Classes EntrainementCycliste et PredictionCycliste definies.");

In [ ]:
// Utilisation des classes
double[] donneesTrajets = new[] { 13.0, 17.0, 20.0, 25.0, 16.0, 11.0, 16.0, 14.0, 12.5 };

// Priors vagues
DonneesCycliste mesDistributions = new DonneesCycliste(
    Gaussian.FromMeanAndPrecision(15, 0.01),  // Prior moyenne
    Gamma.FromShapeAndScale(2, 0.5));         // Prior precision

// Entrainement
EntrainementCycliste entrainement = new EntrainementCycliste();
entrainement.CreationModeleBayesien();
entrainement.DefinirDistributions(mesDistributions);
DonneesCycliste posterieurs = entrainement.CalculePosterieurs(donneesTrajets);

Console.WriteLine($"Moyenne a posteriori : {posterieurs.DistribMoyenne}");
Console.WriteLine($"Precision a posteriori : {posterieurs.DistribBruitTraffic}");

In [ ]:
// Prediction avec les posterieurs
PredictionCycliste prediction = new PredictionCycliste();
prediction.CreationModeleBayesien();
prediction.DefinirDistributions(posterieurs);  // Utiliser les posterieurs comme priors

Gaussian predictionDemain = prediction.EstimerTempsDemain();
Console.WriteLine($"Prediction demain : {predictionDemain}");
Console.WriteLine($"Ecart-type : {Math.Sqrt(predictionDemain.GetVariance()):F2} min");

double probMoins18 = prediction.EstimerTempsDemainInferieurA(18).GetProbTrue();
Console.WriteLine($"P(trajet < 18 min) = {probMoins18:F2}");

## 8. Apprentissage en Ligne

L'apprentissage en ligne permet de mettre a jour le modele incrementalement avec de nouvelles donnees, sans retraiter tout l'historique.

**Principe** : Les posterieurs de la semaine precedente deviennent les priors de la semaine suivante.

In [ ]:
// Nouvelle semaine de donnees
double[] semaineSuivante = new double[] { 18, 25, 30, 14, 11 };

// Utiliser les posterieurs comme nouveaux priors
entrainement.DefinirDistributions(posterieurs);
DonneesCycliste posterieursSemaine2 = entrainement.CalculePosterieurs(semaineSuivante);

Console.WriteLine("=== Apres semaine 2 ===");
Console.WriteLine($"Moyenne a posteriori : {posterieursSemaine2.DistribMoyenne}");
Console.WriteLine($"Precision a posteriori : {posterieursSemaine2.DistribBruitTraffic}");

// Nouvelle prediction
prediction.DefinirDistributions(posterieursSemaine2);
Gaussian nouvellePrediction = prediction.EstimerTempsDemain();
Console.WriteLine($"\nNouvelle prediction demain : {nouvellePrediction}");
Console.WriteLine($"Ecart-type : {Math.Sqrt(nouvellePrediction.GetVariance()):F2} min");

## 9. Probleme : Evenements Extraordinaires

### Observation

Nos donnees contiennent parfois des temps de trajet anormalement longs (25, 30 min) dus a des evenements extraordinaires :
- Accident sur la route
- Meteo extreme
- Travaux

### Solution

Un **modele de melange de Gaussiennes** peut capturer cette bimodalite :
- **Composante 1** : Trajets ordinaires (~15 min)
- **Composante 2** : Trajets extraordinaires (~30 min)

$$p(x) = \pi_1 \cdot \mathcal{N}(x|\mu_1, \tau_1^{-1}) + \pi_2 \cdot \mathcal{N}(x|\mu_2, \tau_2^{-1})$$

ou $\pi_1 + \pi_2 = 1$ sont les poids du melange.

## 10. Modele de Melange de Gaussiennes

In [ ]:
// Structure pour le modele mixte
public struct DonneesCyclisteMixte
{
    public Gaussian[] DistribMoyenne;       // Une par composante
    public Gamma[] DistribBruitTraffic;     // Une par composante
    public Dirichlet DistribMixe;           // Poids du melange
}

// Classe de base pour le modele mixte
public class CyclisteBaseMixte
{
    public InferenceEngine MoteurInference;
    protected int NombreComposantes = 2;
    protected VariableArray<Gaussian> MoyennesAPriori;
    protected VariableArray<Gamma> BruitsAPriori;
    protected Variable<Dirichlet> MixeAPriori;
    protected VariableArray<double> Moyennes;
    protected VariableArray<double> Bruits;
    protected Variable<Vector> Mixe;

    public virtual void CreationModeleBayesien()
    {
        Range indiceComposants = new Range(NombreComposantes);
        
        // VMP est recommande pour les melanges
        MoteurInference = new InferenceEngine(new VariationalMessagePassing());
        MoteurInference.Compiler.CompilerChoice = CompilerChoice.Roslyn;
        
        // Priors pour chaque composante
        MoyennesAPriori = Variable.Array<Gaussian>(indiceComposants);
        BruitsAPriori = Variable.Array<Gamma>(indiceComposants);
        Moyennes = Variable.Array<double>(indiceComposants);
        Bruits = Variable.Array<double>(indiceComposants);
        
        using (Variable.ForEach(indiceComposants))
        {
            Moyennes[indiceComposants] = Variable<double>.Random(MoyennesAPriori[indiceComposants]);
            Bruits[indiceComposants] = Variable<double>.Random(BruitsAPriori[indiceComposants]);
        }
        
        // Prior sur les poids du melange (Dirichlet)
        MixeAPriori = Variable.New<Dirichlet>();
        Mixe = Variable<Vector>.Random(MixeAPriori);
        Mixe.SetValueRange(indiceComposants);
    }

    public virtual void DefinirDistributions(DonneesCyclisteMixte distribsApriori)
    {
        MoyennesAPriori.ObservedValue = distribsApriori.DistribMoyenne;
        BruitsAPriori.ObservedValue = distribsApriori.DistribBruitTraffic;
        MixeAPriori.ObservedValue = distribsApriori.DistribMixe;
    }
}

Console.WriteLine("Classe CyclisteBaseMixte definie.");

In [ ]:
// Classe d'entrainement pour le modele mixte
public class EntrainementCyclisteMixte : CyclisteBaseMixte
{
    protected Variable<int> NombreDeTrajets;
    protected VariableArray<double> TempsDeTrajet;
    protected VariableArray<int> ComposantesTrajets;

    public override void CreationModeleBayesien()
    {
        base.CreationModeleBayesien();
        NombreDeTrajets = Variable.New<int>();
        Range indiceTrajet = new Range(NombreDeTrajets);
        TempsDeTrajet = Variable.Array<double>(indiceTrajet);
        ComposantesTrajets = Variable.Array<int>(indiceTrajet);
        
        using (Variable.ForEach(indiceTrajet))
        {
            // Selection de la composante selon les poids du melange
            ComposantesTrajets[indiceTrajet] = Variable.Discrete(Mixe);
            
            // Variable.Switch selectionne la composante appropriee
            using (Variable.Switch(ComposantesTrajets[indiceTrajet]))
            {
                TempsDeTrajet[indiceTrajet].SetTo(
                    Variable.GaussianFromMeanAndPrecision(
                        Moyennes[ComposantesTrajets[indiceTrajet]], 
                        Bruits[ComposantesTrajets[indiceTrajet]]));
            }
        }
    }

    public DonneesCyclisteMixte CalculePosterieurs(double[] donneesObservees)
    {
        DonneesCyclisteMixte posterieurs;
        NombreDeTrajets.ObservedValue = donneesObservees.Length;
        TempsDeTrajet.ObservedValue = donneesObservees;
        posterieurs.DistribMoyenne = MoteurInference.Infer<Gaussian[]>(Moyennes);
        posterieurs.DistribBruitTraffic = MoteurInference.Infer<Gamma[]>(Bruits);
        posterieurs.DistribMixe = MoteurInference.Infer<Dirichlet>(Mixe);
        return posterieurs;
    }
}

Console.WriteLine("Classe EntrainementCyclisteMixte definie.");

In [ ]:
// Classe de prediction pour le modele mixte
public class PredictionCyclisteMixte : CyclisteBaseMixte
{
    public Variable<double> demainTemps;

    public override void CreationModeleBayesien()
    {
        base.CreationModeleBayesien();
        Variable<int> indiceComposant = Variable.Discrete(Mixe);
        demainTemps = Variable.New<double>();
        using (Variable.Switch(indiceComposant))
        {
            demainTemps.SetTo(Variable.GaussianFromMeanAndPrecision(
                Moyennes[indiceComposant], Bruits[indiceComposant]));
        }
    }

    public Gaussian EstimerTempsDemain()
    {
        return MoteurInference.Infer<Gaussian>(demainTemps);
    }
}

Console.WriteLine("Classe PredictionCyclisteMixte definie.");

## 11. Entrainement du Modele Mixte

In [ ]:
// Donnees avec evenements extraordinaires
double[] donneesMixtes = new[] { 13.0, 17.0, 20.0, 25.0, 16.0, 11.0, 16.0, 25.0, 12.5, 30.0 };

// Priors pour le modele mixte
DonneesCyclisteMixte priorsMMixtes = new DonneesCyclisteMixte
{
    DistribMoyenne = new Gaussian[]
    {
        new Gaussian(15, 100),  // Composante 1 : Ordinaire (~15 min)
        new Gaussian(30, 100)   // Composante 2 : Extraordinaire (~30 min)
    },
    DistribBruitTraffic = new Gamma[]
    {
        Gamma.FromShapeAndScale(2, 0.5),
        Gamma.FromShapeAndScale(2, 0.5)
    },
    DistribMixe = new Dirichlet(1, 1)  // Prior uniforme sur les poids
};

// Entrainement
EntrainementCyclisteMixte entrainementMixte = new EntrainementCyclisteMixte();
entrainementMixte.CreationModeleBayesien();
entrainementMixte.DefinirDistributions(priorsMMixtes);
DonneesCyclisteMixte posterieursMixte = entrainementMixte.CalculePosterieurs(donneesMixtes);

Console.WriteLine("=== Resultats du modele de melange ===");
Console.WriteLine($"\nComposante 1 (Ordinaire):");
Console.WriteLine($"  Moyenne : {posterieursMixte.DistribMoyenne[0]}");
Console.WriteLine($"  Precision : {posterieursMixte.DistribBruitTraffic[0]}");

Console.WriteLine($"\nComposante 2 (Extraordinaire):");
Console.WriteLine($"  Moyenne : {posterieursMixte.DistribMoyenne[1]}");
Console.WriteLine($"  Precision : {posterieursMixte.DistribBruitTraffic[1]}");

Console.WriteLine($"\nPoids du melange : {posterieursMixte.DistribMixe}");
var poidsMoyens = posterieursMixte.DistribMixe.GetMean();
Console.WriteLine($"  -> P(ordinaire) = {poidsMoyens[0]:F2}, P(extraordinaire) = {poidsMoyens[1]:F2}");

In [ ]:
// Prediction avec le modele mixte
PredictionCyclisteMixte predictionMixte = new PredictionCyclisteMixte();
predictionMixte.CreationModeleBayesien();
predictionMixte.DefinirDistributions(posterieursMixte);

Gaussian predictionMixteDemain = predictionMixte.EstimerTempsDemain();
Console.WriteLine($"\nPrediction demain (modele mixte) : {predictionMixteDemain}");
Console.WriteLine($"Ecart-type : {Math.Sqrt(predictionMixteDemain.GetVariance()):F2} min");

## 12. Variable.Switch Explique

### Syntaxe

`Variable.Switch` permet de selectionner dynamiquement une valeur parmi plusieurs options basees sur un indice discret.

```csharp
Variable<int> indice = Variable.Discrete(poids);  // Selectionne 0, 1, 2...

using (Variable.Switch(indice))
{
    // indice est utilise pour selectionner parmi des tableaux
    resultat.SetTo(valeurs[indice]);
}
```

### Applications

- **Modeles de melange** : Selection de la composante
- **Modeles hierarchiques** : Selection du groupe
- **Hidden Markov Models** : Selection de l'etat cache

## 13. Exercice : Melange a 3 Composantes

### Enonce

Modifiez le modele de melange pour utiliser **3 composantes** :
1. Trajets rapides (~10 min)
2. Trajets normaux (~18 min)
3. Trajets longs (~30 min)

### Donnees

```csharp
double[] donnees3Comp = new[] { 8, 10, 12, 18, 17, 19, 20, 28, 32, 35, 11, 18, 30 };
```

### Indice

- Changez `NombreComposantes = 3` dans la classe de base
- Ajustez les priors pour 3 composantes
- Utilisez `Dirichlet(1, 1, 1)` pour les poids

In [ ]:
// EXERCICE : Implementez un melange a 3 composantes

// Modifiez la classe de base pour 3 composantes
public class CyclisteBase3Composantes : CyclisteBaseMixte
{
    public CyclisteBase3Composantes()
    {
        NombreComposantes = 3;  // <- Modification ici
    }
}

public class Entrainement3Composantes : CyclisteBase3Composantes
{
    protected Variable<int> NombreDeTrajets;
    protected VariableArray<double> TempsDeTrajet;
    protected VariableArray<int> ComposantesTrajets;

    public override void CreationModeleBayesien()
    {
        base.CreationModeleBayesien();
        NombreDeTrajets = Variable.New<int>();
        Range indiceTrajet = new Range(NombreDeTrajets);
        TempsDeTrajet = Variable.Array<double>(indiceTrajet);
        ComposantesTrajets = Variable.Array<int>(indiceTrajet);
        
        using (Variable.ForEach(indiceTrajet))
        {
            ComposantesTrajets[indiceTrajet] = Variable.Discrete(Mixe);
            using (Variable.Switch(ComposantesTrajets[indiceTrajet]))
            {
                TempsDeTrajet[indiceTrajet].SetTo(
                    Variable.GaussianFromMeanAndPrecision(
                        Moyennes[ComposantesTrajets[indiceTrajet]], 
                        Bruits[ComposantesTrajets[indiceTrajet]]));
            }
        }
    }

    public DonneesCyclisteMixte CalculePosterieurs(double[] donneesObservees)
    {
        DonneesCyclisteMixte posterieurs;
        NombreDeTrajets.ObservedValue = donneesObservees.Length;
        TempsDeTrajet.ObservedValue = donneesObservees;
        posterieurs.DistribMoyenne = MoteurInference.Infer<Gaussian[]>(Moyennes);
        posterieurs.DistribBruitTraffic = MoteurInference.Infer<Gamma[]>(Bruits);
        posterieurs.DistribMixe = MoteurInference.Infer<Dirichlet>(Mixe);
        return posterieurs;
    }
}

// Donnees
double[] donnees3Comp = new[] { 8.0, 10.0, 12.0, 18.0, 17.0, 19.0, 20.0, 28.0, 32.0, 35.0, 11.0, 18.0, 30.0 };

// Priors pour 3 composantes
DonneesCyclisteMixte priors3Comp = new DonneesCyclisteMixte
{
    DistribMoyenne = new Gaussian[]
    {
        new Gaussian(10, 100),  // Rapide
        new Gaussian(18, 100),  // Normal
        new Gaussian(30, 100)   // Long
    },
    DistribBruitTraffic = new Gamma[]
    {
        Gamma.FromShapeAndScale(2, 0.5),
        Gamma.FromShapeAndScale(2, 0.5),
        Gamma.FromShapeAndScale(2, 0.5)
    },
    DistribMixe = new Dirichlet(1, 1, 1)  // Uniforme sur 3 composantes
};

// Entrainement
Entrainement3Composantes ent3 = new Entrainement3Composantes();
ent3.CreationModeleBayesien();
ent3.DefinirDistributions(priors3Comp);
DonneesCyclisteMixte post3 = ent3.CalculePosterieurs(donnees3Comp);

Console.WriteLine("=== Modele a 3 composantes ===");
for (int i = 0; i < 3; i++)
{
    Console.WriteLine($"\nComposante {i+1}: Moyenne = {post3.DistribMoyenne[i].GetMean():F1} min");
}
Console.WriteLine($"\nPoids : {post3.DistribMixe.GetMean()}");

## 14. Resume

| Concept | Description |
|---------|-------------|
| **Gaussian** | Distribution continue pour les valeurs reelles |
| **Gamma** | Prior conjugue pour la precision d'une Gaussienne |
| **Dirichlet** | Prior conjugue pour les poids d'un melange |
| **Variable.Switch** | Selection dynamique parmi plusieurs options |
| **Apprentissage en ligne** | Posterieurs -> Priors pour nouvelles donnees |
| **Modele de melange** | Capture les distributions multimodales |

---

## Prochaine etape

Dans [Infer-3-Factor-Graphs](Infer-3-Factor-Graphs.ipynb), nous explorerons :

- Les graphes de facteurs et leur representation
- L'inference discrete avec le probleme du "Murder Mystery"
- Le paradoxe de Monty Hall
- Le theoreme de Bayes illustre